# WildlifeReID-10k creation - part 1

This is the first part for creating the WildlifeReID-10k dataset. It copies the files to a separate folder, applies bounding boxes and masks and combined them together. The split is created in the second part.

First load the necessary packages.

In [1]:
import sys
sys.path.insert(0, '..')

import os
import pandas as pd
from prepare_wildlife_reid_10k import prepare_functions, species_conversion

Then specify the roots, where the dataset is located. Parameters `size` specifies the size to which the datasets will be resized and parameters `copy_files` whether the files copied from `root_datasets` to `root`. Since bounding boxes and masks are applied and the black borders are cropped, it is relatively time-consuming.

In [2]:
root_datasets = '/data/wildlife_datasets/data'
root = os.path.join(root_datasets, 'WildlifeReID10k')
root_images = os.path.join(root, 'images')
root_metadata = os.path.join(root, 'metadata')
size = None
copy_files = False

Create metadata for each dataset and potentially copy the files.

In [3]:
for name, prepare in prepare_functions.items():
    print(name)
    os.makedirs(f'{root_metadata}/{name}/', exist_ok=True)
    metadata_part = prepare(size=size, root=f'{root_datasets}/{name}', new_root=f'{root_images}/{name}', copy_files=copy_files)
    metadata_part.to_csv(f'{root_metadata}/{name}/metadata.csv', index=False)

AAUZebraFish


100%|████████████████████████████████████████████████████████| 6672/6672 [00:00<00:00, 11505.84it/s]


AerialCattle2017


100%|████████████████████████████████████████████████████████| 4700/4700 [00:00<00:00, 12358.95it/s]


ATRW


...

100%|██████████████████████████████████████████████████████████| 820/820 [00:00<00:00, 21802.68it/s]


WhaleSharkID


100%|████████████████████████████████████████████████████████| 7693/7693 [00:00<00:00, 13954.32it/s]


ZindiTurtleRecall


100%|██████████████████████████████████████████████████████| 12803/12803 [00:00<00:00, 20280.86it/s]


The next codes adds additional information to the metadata and combines them together. After this code, the dataset is finished but splits. To compute splits, we first need to compute the features from `extract_features.py` and then compute the actual splits in `prepare_wildlife_reid_10k_2.ipynb`.

In [4]:
metadata = []
for name in prepare_functions:
    metadata_part = pd.read_csv(f'{root_metadata}/{name}/metadata.csv')
    metadata_part['dataset'] = name
    metadata_part['identity'] = name + '_' + metadata_part['identity'].astype(str)
    metadata_part['path'] = 'images/' + name + '/' + metadata_part['path']
    metadata_part['species'] = metadata_part['species'].apply(lambda x: species_conversion[x])
    metadata.append(metadata_part)
metadata = pd.concat(metadata).reset_index(drop=True)
metadata = metadata.drop('image_id', axis=1)
metadata['image_id'] = range(len(metadata))
idx = ~metadata['date'].isnull()
idx = metadata.index[idx]
metadata.loc[idx, 'date'] = pd.to_datetime(metadata.loc[idx, 'date'].astype(str).apply(lambda x: x[:10]), format='%Y-%m-%d')
metadata['orientation'] = metadata['orientation'].replace({'below': 'down', 'up': 'top', 'above': 'top'})
metadata.to_csv(f'{root}/metadata.csv', index=False)